In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import pandas as pd



In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [3]:
data_original = pd.read_csv('Pressupostos_dels_ens_municipals_de_Catalunya.csv')
data_original.head(6)

,ANY_EXERCICI,TIPUS_PARTIDA,TIPUS_CLASSIF,CODI_PANTALLA,NIVELL,DESCRIPCIO,IMPORT,ID_ESTRUCTURA,ESTRUCTURA,CODI_ENS,NOM_COMPLERT
0,01/01/2016 12:00:00 AM,I,E,3.0,1,"Taxes, preus públics i altres ingressos",20.0,7,3,9820170005,Agència Local d'Energia de Barcelona
1,01/01/2016 12:00:00 AM,I,E,36.0,2,Vendes,8.0,7,36,9820170005,Agència Local d'Energia de Barcelona
2,01/01/2016 12:00:00 AM,I,E,38.0,2,Reintegraments d'operacions corrents,6.0,7,38,9820170005,Agència Local d'Energia de Barcelona
3,01/01/2016 12:00:00 AM,I,E,389.0,3,Altres reintegraments d'operacions corrents,6.0,7,3809,9820170005,Agència Local d'Energia de Barcelona
4,01/01/2016 12:00:00 AM,I,E,39.0,2,Altres ingressos,6.0,7,39,9820170005,Agència Local d'Energia de Barcelona
5,01/01/2016 12:00:00 AM,I,E,399.0,3,Altres ingressos diversos,6.0,7,3909,9820170005,Agència Local d'Energia de Barcelona


In [4]:
data_16 = data_original.loc[data_original.ANY_EXERCICI == '01/01/2016 12:00:00 AM']
data_16.shape

(624092, 11)

In [5]:
data_16_aj = data_16.loc[data_16.NOM_COMPLERT.str.contains('Ajuntament')]
data_16_aj.shape

(434340, 11)

In [6]:
data_16_aj_no0=data_16_aj.loc[data_16_aj.IMPORT > 0.0]

In [7]:
name, counts = np.unique(data_16_aj_no0.NOM_COMPLERT, return_counts = True)
print(np.amax(counts), np.amin(counts), np.mean(counts), np.median(counts))


387 47 201.99092970521542 193.0


In [8]:
np.unique(data_16_aj_no0.TIPUS_PARTIDA)


array(['D', 'I'], dtype=object)

In [9]:
np.unique(data_16_aj_no0.TIPUS_CLASSIF)


array(['E', 'F'], dtype=object)

In [10]:
np.unique(data_16_aj_no0.NIVELL)


array([1, 2, 3, 4])

In [11]:
def description_by_level_despesa (data, level):
    desc, count = np.unique(data.loc[(data.NIVELL == level) & (data.TIPUS_PARTIDA == 'D'), ['DESCRIPCIO']], return_counts = True)
    return dict(zip(desc,counts))

res = description_by_level_despesa(data_16_aj_no0, 1)
print("length: ", len(res))
res

length:  15


{'Actius financers': 249,
 'Actuacions de caràcter econòmic': 230,
 'Actuacions de caràcter general': 257,
 'Actuacions de protecció i promoció social': 157,
 'Despeses corrents en béns i serveis': 205,
 'Despeses de personal': 204,
 'Despeses financeres': 172,
 'Deute públic': 270,
 'Fons de contingència i altres imprevistos': 122,
 'Inversions reals': 114,
 'Passius financers': 209,
 'Producció de béns públics de caràcter preferent': 218,
 'Serveis públics bàsics': 147,
 'Transferències corrents': 270,
 'Transferències de capital': 122}

In [12]:
data_sorted=data_16_aj_no0[['NOM_COMPLERT','CODI_ENS','TIPUS_PARTIDA','TIPUS_CLASSIF','CODI_PANTALLA', 'NIVELL', 'IMPORT','DESCRIPCIO','ESTRUCTURA']].astype(str)

data_sorted.sort_values(['CODI_ENS','TIPUS_PARTIDA','TIPUS_CLASSIF','CODI_PANTALLA'])
    

,NOM_COMPLERT,CODI_ENS,TIPUS_PARTIDA,TIPUS_CLASSIF,CODI_PANTALLA,NIVELL,IMPORT,DESCRIPCIO,ESTRUCTURA
4011,Ajuntament d'Agullana,1700100000,D,E,1.0,1,287505.72,Despeses de personal,1
4012,Ajuntament d'Agullana,1700100000,D,E,10.0,2,17505.72,Òrgans de govern i personal directiu,10
4013,Ajuntament d'Agullana,1700100000,D,E,100.0,3,17505.72,Retr. bàs. i altres rem. membres òrgans de govern,1000
4014,Ajuntament d'Agullana,1700100000,D,E,100.0,4,17505.72,Retribucions bàsiques,100000
4015,Ajuntament d'Agullana,1700100000,D,E,12.0,2,38000.0,Personal funcionari,12
4016,Ajuntament d'Agullana,1700100000,D,E,120.0,3,12000.0,Retribucions bàsiques,1200
4017,Ajuntament d'Agullana,1700100000,D,E,120.0,4,12000.0,Sous del grup A1,120000
4018,Ajuntament d'Agullana,1700100000,D,E,121.0,3,26000.0,Retribucions complementàries,1201
4019,Ajuntament d'Agullana,1700100000,D,E,121.0,4,26000.0,Complement de destinació,120100
4020,Ajuntament d'Agullana,1700100000,D,E,13.0,2,165000.0,Personal laboral,13


###### Will you need to transform data in a some way in order to be able to compare different budgets? 
YES
##### How?
Actualment tenim les dades organitzades per ajuntament, però per millorar la visió global del presupost hem pensat d'agrupar tots els nivells 1 de cada ajuntament, separtant els Ingressos i les Despeses

In [16]:
def get_import_ajuntament(data, codi_ens, tipus):
    return np.sum(data.loc[(data.CODI_ENS == codi_ens) & (data.TIPUS_PARTIDA == tipus), ['IMPORT']].IMPORT)

def get_ingressos_despesa_per_ajuntament(data):
    codi_ens = np.unique(data.CODI_ENS)
    nom = np.unique(data.NOM_COMPLERT)



    data_comparative = pd.DataFrame(columns=['NOM_COMPLERT','CODI_ENS', 'DESPESES','INGRESSOS'])
    
    i = 0
    for ens in codi_ens:
      
        despesses = get_import_ajuntament(data, ens, 'D')
        ingressos = get_import_ajuntament(data, ens, 'I')
        data_comparative.loc[i] = [nom[i], ens, despesses,ingressos]
        print(data_comparative.loc[i])
        i = i +1
        
    return data_comparative


ingres_despesa = get_ingressos_despesa_per_ajuntament(data_sorted)




        

NOM_COMPLERT                                  Ajuntament d'Abrera
CODI_ENS                                               1700100000
DESPESES        67600.067600.067600.0238265.5644250.020000.022...
INGRESSOS       269899.59265945.451867.21195692.9363385.315000...
Name: 0, dtype: object
NOM_COMPLERT                                Ajuntament d'Agramunt
CODI_ENS                                               1700250006
DESPESES        7000.040000.019000.0259004.083204.012000.05120...
INGRESSOS       873000.0513000.08000.0403000.087000.015000.036...
Name: 1, dtype: object
NOM_COMPLERT                      Ajuntament d'Aguilar de Segarra
CODI_ENS                                               1700310007
DESPESES        2300.02300.02300.027980.010200.010200.017780.0...
INGRESSOS       48360.045810.03090.032000.08670.02050.02550.02...
Name: 2, dtype: object
NOM_COMPLERT                                Ajuntament d'Agullana
CODI_ENS                                               1700460009
DESPESE

NOM_COMPLERT                                 Ajuntament d'Almenar
CODI_ENS                                               1703140003
DESPESES        136723.3213700.03700.010000.015000.015000.0150...
INGRESSOS       88779.8278279.828341.1450331.3819107.3500.0105...
Name: 29, dtype: object
NOM_COMPLERT                                Ajuntament d'Almoster
CODI_ENS                                               1703290004
DESPESES        497233.0497233.0497233.02973379.0651007.065100...
INGRESSOS       2532600.02492600.02600.02130000.0150000.021000...
Name: 30, dtype: object
NOM_COMPLERT                                     Ajuntament d'Alp
CODI_ENS                                               1703350006
DESPESES        617000.0617000.0617000.03655075.26543786.45375...
INGRESSOS       3635500.03135500.033500.02500000.0402000.02000...
Name: 31, dtype: object
NOM_COMPLERT                                  Ajuntament d'Alpens
CODI_ENS                                               1703400000
DESP

NOM_COMPLERT                                 Ajuntament d'Espolla
CODI_ENS                                               1706690004
DESPESES        6738000.06738000.06738000.022331125.925039526....
INGRESSOS       23643500.021893500.033500.016860000.02900000.0...
Name: 61, dtype: object
NOM_COMPLERT                               Ajuntament d'Esponellà
CODI_ENS                                               1706750006
DESPESES        256687.02700.02700.086887.06460.03100.02800.02...
INGRESSOS       405200.0381700.027000.0264300.063400.027000.02...
Name: 62, dtype: object
NOM_COMPLERT                                   Ajuntament d'Espot
CODI_ENS                                               1706940003
DESPESES        174000.078000.078000.031000.047000.096000.0300...
INGRESSOS       25000.0485880.0479880.07000.0420800.025580.026...
Name: 63, dtype: object
NOM_COMPLERT                               Ajuntament d'Estamariu
CODI_ENS                                               1707080001
DESP

NOM_COMPLERT                          Ajuntament d'Os de Balaguer
CODI_ENS                                               1710580001
DESPESES        80244.610000.010000.056100.01000.0600.028500.0...
INGRESSOS       98528.4496528.4424206.3148297.7422024.392000.0...
Name: 91, dtype: object
NOM_COMPLERT                                    Ajuntament d'Osor
CODI_ENS                                               1710610007
DESPESES        70000.070000.070000.038900.023500.023500.01540...
INGRESSOS       68778.1468268.144748.2953103.68916.251500.0510...
Name: 92, dtype: object
NOM_COMPLERT                             Ajuntament d'Ossó de Sió
CODI_ENS                                               1710770005
DESPESES        29000.029000.029000.0124000.015000.015000.0150...
INGRESSOS       233100.0228100.01100.0194000.025000.08000.0500...
Name: 93, dtype: object
NOM_COMPLERT                               Ajuntament d'Ullastret
CODI_ENS                                               1710960009
DESP

NOM_COMPLERT                     Ajuntament de Bell-lloc d'Urgell
CODI_ENS                                               1713900000
DESPESES        3000.03000.03000.0389266.8222080.020000.02080....
INGRESSOS       916369.19780335.782919.65620416.13150000.07000...
Name: 120, dtype: object
NOM_COMPLERT                            Ajuntament de Bellaguarda
CODI_ENS                                               1714040003
DESPESES        227499.41227499.41227499.412514861.63451218.48...
INGRESSOS       60150.961485480.01451880.01980.01265000.062900...
Name: 121, dtype: object
NOM_COMPLERT                    Ajuntament de Bellcaire d'Empordà
CODI_ENS                                               1714260009
DESPESES        104200.0104200.0104200.01064270.3455900.051900...
INGRESSOS       1185500.01060500.04500.0760000.0265000.031000....
Name: 122, dtype: object
NOM_COMPLERT                      Ajuntament de Bellmunt d'Urgell
CODI_ENS                                               1714320002
D

...

NOM_COMPLERT                                Ajuntament de Vinaixa
CODI_ENS                                                816910007
DESPESES        2986446.02986446.02986446.031528376.07041619.0...
INGRESSOS       59803774.052061281.08500.015500000.029244628.0...
Name: 754, dtype: object
NOM_COMPLERT                                Ajuntament de Vinebre
CODI_ENS                                                817120002
DESPESES        65440.065440.065440.0447060.03900.0900.03000.0...
INGRESSOS       985000.0933000.016500.0700000.01500.0197000.01...
Name: 755, dtype: object
NOM_COMPLERT                     Ajuntament de Vinyols i els Arcs
CODI_ENS                                                817270005
DESPESES        1600349.851600349.851600349.859219400.34247362...
INGRESSOS       8701726.748401726.741516.586500000.01350000.05...
Name: 756, dtype: object
NOM_COMPLERT                                  Ajuntament de Xerta
CODI_ENS                                                817480001
D

NOM_COMPLERT                               Ajuntament de la Febró
CODI_ENS                                                820210007
DESPESES        1637067.691637067.691637067.695995867.45147186...
INGRESSOS       8764939.817319853.4527381.575807303.07966034.1...
Name: 783, dtype: object
NOM_COMPLERT                             Ajuntament de la Figuera
CODI_ENS                                                820370005
DESPESES        293300.0293300.0293300.01941436.0397913.038691...
INGRESSOS       2679800.02652700.021500.02175000.01200.0230000...
Name: 784, dtype: object
NOM_COMPLERT                            Ajuntament de la Floresta
CODI_ENS                                                820420002
DESPESES        314500.0314500.0314500.01341740.0510655.039525...
INGRESSOS       2050100.02020100.03100.01600000.0267000.015000...
Name: 785, dtype: object
NOM_COMPLERT                              Ajuntament de la Galera
CODI_ENS                                                820550006
D

NOM_COMPLERT                             Ajuntament de la Secuita
CODI_ENS                                                823460009
DESPESES        374862.79374862.79374862.791506342.17351772.19...
INGRESSOS       30000.02021210.571911210.576237.511567973.0630...
Name: 813, dtype: object
NOM_COMPLERT                        Ajuntament de la Selva de Mar
CODI_ENS                                                823590004
DESPESES        661014.53661014.53661014.533928350.11266174.17...
INGRESSOS       4075000.03990000.013000.03125000.0302000.05500...
Name: 814, dtype: object
NOM_COMPLERT                       Ajuntament de la Sentiu de Sió
CODI_ENS                                                823620002
DESPESES        660828.9449303.9743253.973000.03050.0186994.77...
INGRESSOS       992600.0687600.03600.0545000.0129000.010000.03...
Name: 815, dtype: object
NOM_COMPLERT                        Ajuntament de la Seu d'Urgell
CODI_ENS                                                823780001
D

NOM_COMPLERT                    Ajuntament de les Valls de Valira
CODI_ENS                                                826870005
DESPESES        23500.0127400.035200.018200.035000.02000.01120...
INGRESSOS       385.015000.090000.0200.07500.0200.01000.057300...
Name: 842, dtype: object
NOM_COMPLERT                                  Ajuntament del Bruc
CODI_ENS                                                826900000
DESPESES        31200.0137500.0137500.0137500.0840800.095700.0...
INGRESSOS       2100000.02040000.05000.01600000.0235000.020000...
Name: 843, dtype: object
NOM_COMPLERT                                 Ajuntament del Brull
CODI_ENS                                                827040003
DESPESES        17083754.994782000.04782000.04782000.04229308....
INGRESSOS       25390000.024190000.060000.017100000.01330000.0...
Name: 844, dtype: object
NOM_COMPLERT                               Ajuntament del Catllar
CODI_ENS                                                827110007
D

NOM_COMPLERT                              Ajuntament del Vilosell
CODI_ENS                                                830200000
DESPESES        950521.72950521.72950521.723864570.7879506.778...
INGRESSOS       5000.04045000.03945000.015000.03085000.0720000...
Name: 871, dtype: object
NOM_COMPLERT                               Ajuntament dels Alamús
CODI_ENS                                                830360009
DESPESES        631613.09488720.444000.02500.02500.0482220.444...
INGRESSOS       288200.0229200.03200.075000.0124000.026000.010...
Name: 872, dtype: object
NOM_COMPLERT                            Ajuntament dels Garidells
CODI_ENS                                                830410007
DESPESES        293330.0125200.090200.021500.013500.013500.011...
INGRESSOS       564500.0454500.05500.0340000.084000.025000.011...
Name: 873, dtype: object
NOM_COMPLERT                 Ajuntament dels Hostalets de Pierola
CODI_ENS                                                830540003
D

In [14]:
print(ingres_despesa)

                                          NOM_COMPLERT    CODI_ENS  \
0                                  Ajuntament d'Abrera  1700100000   
1                                Ajuntament d'Agramunt  1700250006   
2                      Ajuntament d'Aguilar de Segarra  1700310007   
3                                Ajuntament d'Agullana  1700460009   
4                              Ajuntament d'Aiguafreda  1700590004   
5                             Ajuntament d'Aiguamúrcia  1700620002   
6                               Ajuntament d'Aiguaviva  1700780001   
7                                  Ajuntament d'Aitona  1700840003   
8                                 Ajuntament d'Albanyà  1700970005   
9                              Ajuntament d'Albatàrrec  1701010007   
10                                 Ajuntament d'Albesa  1701180001   
11                              Ajuntament d'Albinyana  1701230008   
12                                 Ajuntament d'Albons  1701390004   
13                  